In [36]:
import os
import json
from PIL import Image

import torch
import torch.utils.data as data
import torchvision.transforms.v2 as tfs
import torch.nn as nn
import torch.optim as optim

from tqdm import tqdm

In [35]:
class DigitDataset(data.Dataset):
    def __init__(self, path, train=True, transform=None):
        self.path = os.path.join(path, 'train' if train else 'test')
        self.transform = transform

        with open(os.path.join(path, 'format.json'), 'r') as fp:
            self.format = json.load(fp)
        
        self.length = 0
        self.files = []
        self.targets = torch.eye(10)

        for _dir, _target in self.format.items():
            path = os.path.join(self.path, _dir)
            list_files = os.listdir(path)
            self.length += len(list_files)
            self.files.extend(map(lambda _x: (os.path.join(path, _x), _target), list_files))

    def __getitem__(self, item):
        path_file, target = self.files[item]
        t = self.targets[target]
        img = Image.open(path_file)

        if self.transform:
            img = self.transform(img).ravel().float() / 255.0
        return img, t
    
    def __len__(self):
        return self.length

In [34]:
class DigitNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(28*28, 32)
        self.layer2 = nn.Linear(32, 10)

    def forward(self, x):
        x = self.layer1(x)
        x = nn.functional.relu(x)
        x = self.layer2(x)
        return x

In [38]:
model = DigitNN()

to_tensor = tfs.ToImage()
d_train = DigitDataset(path='dataset', transform=to_tensor)
train_data = data.DataLoader(d_train, batch_size=32, shuffle=True)

optimizer = optim.Adam(model.parameters(), lr=0.01)
loss_function = nn.CrossEntropyLoss()

epochs = 2
model.train()

for epoch in range(epochs):
    loss_mean = 0
    lm_count = 0

    train_tqdm = tqdm(train_data, leave=True)

    for x_train, y_train in train_tqdm:
        predict = model(x_train)
        loss = loss_function(predict, y_train)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        lm_count += 1
        loss_mean = 1 / lm_count * loss.item() + (1 - 1 / lm_count) * loss_mean
        train_tqdm.set_description(f'Epoch {epoch+1}/{epochs} Loss: {loss_mean:.4f}')

Epoch 2/2 Loss: 0.1893: 100%|██████████| 1875/1875 [00:23<00:00, 81.04it/s]


In [39]:
d_test = DigitDataset(path='dataset', train=False, transform=to_tensor)
test_data = data.DataLoader(d_test, batch_size=500, shuffle=False)
Q = 0

model.eval()
for x_test, y_test in test_data:
    with torch.no_grad():
        predict = model(x_test)
        predict = torch.argmax(predict, dim=1)
        y = torch.argmax(y_test, dim=1)
        Q += torch.sum(predict == y).item()

Q /= len(d_test)
Q    

0.9463